In [0]:
import numpy as np
import pandas as pd
import datetime
import random

# Dataset

In [0]:
from google.colab import files
uploaded=files.upload()

Saving VDS_Gardiner VDS_NO.89_log.csv to VDS_Gardiner VDS_NO.89_log.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


The Columns in Detail:

1st Column: Date/ time of the feed (Time Stamp)

2nd Column: Unique continuous ID for each detector base on the road direction (easy for subscription)

3rd Column: Detector Name in the system

4th Column: Detector physical address

5th Column: Detector's Latitude

6th Column: Detector's Longitude

7th Column: Integer represnting the lane at which vehicles are being detected (Lane Index)

8th Column: % of time loop is occupied per interval

9th Column: Average speed during an interval (km/h)

10th Column: vehicles per interval in (100 vehicle/hour)

11th Column: VdsDevice's unique ID

12th Colunn: Region Name where the detector is located

In [0]:
data = pd.read_csv("VDS_Gardiner VDS_NO.89_log.csv")
data = data[data['onerHourBefore100VehiclesPerHour'].notna()]
data.index = pd.to_datetime(data['timeStamp'])
data['Hour'] = data.index.hour
data['Minute'] = data.index.minute
del data['timeStamp']
del data['detectorID']
data = data.reset_index(drop = True)
data.loc[data.dayOfWeek == 'Mon','dayOfWeek'] = 1
data.loc[data.dayOfWeek == 'Tue','dayOfWeek'] = 2
data.loc[data.dayOfWeek == 'Wed','dayOfWeek'] = 3
data.loc[data.dayOfWeek == 'Thu','dayOfWeek'] = 4
data.loc[data.dayOfWeek == 'Fri','dayOfWeek'] = 5
data.loc[data.dayOfWeek == 'Sat','dayOfWeek'] = 6
data.loc[data.dayOfWeek == 'Sun','dayOfWeek'] = 7
data[0:10]

,averageSpeed,100VehiclesPerHour,dayOfWeek,onerHourBefore100VehiclesPerHour,Hour,Minute
0,0,0,1,1.0,1,0
1,0,0,1,0.0,1,2
2,67,1,1,3.0,1,4
3,106,1,1,0.0,1,6
4,0,0,1,5.0,1,8
5,76,1,1,1.0,1,10
6,62,1,1,0.0,1,12
7,0,0,1,1.0,1,14
8,0,0,1,1.0,1,16
9,0,0,1,3.0,1,18


#Different Data

In [9]:
!unzip /content/drive/My\ Drive/dot_traffic_2015.txt.gz.zip

Archive:  /content/drive/My Drive/dot_traffic_2015.txt.gz.zip
  inflating: dot_traffic_2015.txt.gz  


In [0]:
traffic_df = pd.read_csv('/content/dot_traffic_2015.txt.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [15]:
traffic_df.head(10)

,day_of_data,day_of_week,direction_of_travel,direction_of_travel_name,fips_state_code,functional_classification,functional_classification_name,lane_of_travel,month_of_data,record_type,restrictions,station_id,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400,year_of_data
0,7,3,1,North,56,3R,Rural: Principal Arterial - Other,1,4,3,NaN,000084,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6,15
1,26,7,7,West,21,1U,Urban: Principal Arterial - Interstate,2,9,3,NaN,056P94,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613,15
2,16,3,3,East,6,1U,Urban: Principal Arterial - Interstate,0,6,3,NaN,077590,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892,15
3,26,1,5,South,55,1U,Urban: Principal Arterial - Interstate,1,4,3,NaN,450001,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110,15
4,23,7,3,East,4,4R,Rural: Minor Arterial,0,5,3,NaN,102210,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7,15
5,25,7,5,South,44,2U,Urban: Principal Arterial - Other Freeways or ...,0,7,3,NaN,008148,1168,781,738,425,279,395,715,1202,1486,1819,2193,2401,2691,2508,2700,2673,2746,2564,2257,2140,2060,2156,1873,1590,15
6,10,5,3,East,19,3U,Urban: Principal Arterial - Other,2,9,3,NaN,000823,34,43,27,48,88,269,334,476,444,355,375,434,492,499,539,542,571,559,395,351,262,217,153,61,15
7,27,3,3,East,36,4U,Urban: Minor Arterial,1,10,3,NaN,009582,13,3,8,3,17,47,95,158,151,151,141,184,193,219,217,238,278,250,154,130,84,56,21,14,15
8,26,6,7,West,42,1R,Rural: Principal Arterial - Interstate,2,6,3,NaN,000393,115,78,31,40,42,134,268,335,392,441,600,722,652,706,751,740,784,822,703,425,192,159,150,168,15
9,12,3,5,South,40,2U,Urban: Principal Arterial - Other Freeways or ...,2,5,3,NaN,AVC068,189,100,115,108,219,527,1212,1596,1347,1115,1114,1126,1178,1193,1364,1662,1673,1519,1148,914,808,555,468,270,15


In [0]:
del traffic_df['functional_classification_name']

In [0]:
set(traffic_df['restrictions'])


#DataLoader

In [0]:
catcols = ["averageSpeed",	"100VehiclesPerHour",	"dayOfWeek",	"onerHourBefore100VehiclesPerHour",	"Hour",	"Minute"]

In [0]:
np.random.seed(50) # set the numpy seed for consistent split
arr = np.arange(len(data))
np.random.shuffle(arr)
np_data=data.values
shuffled_data=np.array([np_data[x,:] for x in arr])
total_data=len(shuffled_data)
train_data=shuffled_data[:int(total_data*0.7)].astype(np.float32)
validation_data=shuffled_data[int(total_data*0.7):int(total_data*0.85)].astype(np.float32)
test_data=shuffled_data[int(total_data*0.85):].astype(np.float32)
print("Training dataset size: {}".format(train_data.shape[0]))
print("Validation dataset size: {}".format(validation_data.shape[0]))
print("Test dataset size: {}".format(test_data.shape[0]))

Training dataset size: 4200
Validation dataset size: 900
Test dataset size: 900


#Neural Network

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

In [0]:
cuda = torch.device('cuda')

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch, momentum):
  """ Generate a name for the model consisting of all the hyperparameter values
  Args:
  config: Configuration object containing the hyperparameters
  Returns:
  path: A string with the hyperparameter name and value concatenated
  """
  if momentum:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}_momentum{4}".format(name,
            batch_size,
            learning_rate,
            epoch,momentum)
  else:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
            batch_size,
            learning_rate,
            epoch)
  return path

In [0]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.encoder = nn.Sequential(
    nn.Linear(6, 18), # TODO
    nn.ReLU(),
    nn.Linear(18, 32),
    nn.ReLU(),
    nn.Linear(32, 64),
    nn.ReLU(),
    )
    self.decoder = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 6),# TODO
    nn.Sigmoid() # get to the range (0, 1)
    )
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x

In [0]:
cat_index = {}  # Mapping of feature -> start index of feature in a record
cat_values = {} # Mapping of feature -> list of categorical values the feature can take

# build up the cat_index and cat_values dictionary
for i, header in enumerate(data.keys()):
    if "_" in header: # categorical header
        feature, value = header.split()
        feature = feature[:-1] # remove the last char; it is always an underscore
        if feature not in cat_index:
            cat_index[feature] = i
            cat_values[feature] = [value]
        else:
            cat_values[feature].append(value)
def get_onehot(record, feature):
    """
    Return the portion of `record` that is the one-hot encoding
    of feature. For example, since the feature "work" is stored
    in the indices [5:12] in each record, calling `get_range(record, "work")`
    is equivalent to accessing `record[5:12]`.
    
    Args:
        - record: a numpy array representing one record, formatted
                  the same way as a row in `data.np`
        - feature: a string, should be an element of `catcols`
    """
    start_index = cat_index[feature]
    stop_index = cat_index[feature] + len(cat_values[feature])
    return record[start_index:stop_index]

def get_categorical_value(onehot, feature):
    """
    Return the categorical value name of a feature given
    a one-hot vector representing the feature.
    
    Args:
        - onehot: a numpy array one-hot representation of the feature
        - feature: a string, should be an element of `catcols`
        
    Examples:
    
    >>> get_categorical_value(np.array([0., 0., 0., 0., 0., 1., 0.]), "work")
    'State-gov'
    >>> get_categorical_value(np.array([0.1, 0., 1.1, 0.2, 0., 1., 0.]), "work")
    'Private'
    """
    # TODO
    idx = np.argmax(onehot)
    return cat_values[feature][idx]
def get_feature(record, feature):
    """
    Return the categorical feature value of a record
    """
    onehot = get_onehot(record, feature)
    return get_categorical_value(onehot, feature)

def get_features(record):
    """
    Return a dictionary of all categorical feature values of a record
    """
    return { f: get_feature(record, f) for f in catcols }

In [0]:
def get_accuracy(model, data_loader):
    """Return the "accuracy" of the autoencoder model across a data set
    
    Args:
       - model: the autoencoder model, an instance of nn.Module
       - data_loader: an instance of torch.utils.data.DataLoader

    Example (to illustrate how get_accuracy is intended to be called.
             depending on your variable naming this code might not work
             out of the box)

        >>> model = AutoEncoder()
        >>> vdl = torch.utils.data.DataLoader(data_valid, batch_size=256, shuffle=True)
        >>> get_accuracy(model, vdl)
    """
    total = 0
    acc = 0
    for col in catcols:
        for item in data_loader: # minibatches
            inp = item.detach().numpy()
            out = model(zero_out_feature(item.clone())).detach().numpy()
            for i in range(out.shape[0]): # record in minibatch
                acc += np.square(out[i][1] - inp[i][1] )
    return acc 

In [0]:
import matplotlib.pyplot as plt
def plot_training_curve(path):
    train_err = np.loadtxt("{}_train_acc.csv".format(path))
    val_err = np.loadtxt("{}_val_acc.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Accuracy")
    plt.plot(range(1,train_err.shape[0]+1), train_err, label="Train")
    plt.plot(range(1,val_err.shape[0]+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,train_loss.shape[0]+1), train_loss, label="Train")
    plt.plot(range(1,val_loss.shape[0]+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()
def zero_out_feature(records):
    """ Set the feature missing in records, by setting the appropriate
    columns of records to 0
    """
    records[:, 1] = 0
    return records

def zero_out_random_feature(records):
    """ Set one random feature missing in records, by setting the 
    appropriate columns of records to 0
    """
    return zero_out_feature(records)
def validation_eval(model, valid_loader, criterion):
    valid_error = 0
    total_valid_loss = 0
    for data_val in valid_loader:
        recon=model(data_val)
        loss = criterion(recon, data_val)
        total_valid_loss += loss.item()
    return total_valid_loss/len(valid_loader),get_accuracy(model, valid_loader)
        
def train(model, train_loader, valid_loader, num_epochs=5, learning_rate=1e-4, batch_size=1,betas=(0.9, 0.999)):
    """ Training loop. You should update this."""
    torch.manual_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,betas=betas)
    train_err = []    
    train_loss = []  
    val_err = []  
    val_loss = []
    for epoch in range(num_epochs):
        train_error = 0
        total_train_loss = 0
        train_idx = 0
        for data in train_loader:
            data_zero = zero_out_feature(data.clone()) # zero out one categorical feature
            recon = model(data_zero)
            loss = criterion(recon, data)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_train_loss += loss.item()
            if train_idx%1000==0:
                train_err.append(get_accuracy(model,  train_loader))
                train_loss.append(total_train_loss/(train_idx+1))
                valid_loss,valid_err = validation_eval(model, valid_loader,criterion)
                val_loss.append(valid_loss)
                val_err.append(valid_err)
            train_idx+=1
        train_loss.append(total_train_loss/len(train_loader))
        valid_loss,_= validation_eval(model, valid_loader,criterion)
        val_loss.append(valid_loss)
        print(("Epoch {}: Train loss: {} |"+               
               "Validation loss: {}").format(
            epoch + 1, train_loss[-1],val_loss[-1]))
        model_path = "Encoder_Decoder_Batchsize{}_lr{}_epoch{}_betas{}".format(batch_size,learning_rate,epoch,betas)
        torch.save(model.state_dict(), model_path)
    print('Finished Training')    
    np.savetxt("{}_train_loss.csv".format(model_path), np.array(train_loss))       
    np.savetxt("{}_val_loss.csv".format(model_path), np.array(val_loss))
    return model_path


In [0]:
Encoder=AutoEncoder()
batch_size=32
train_loader =torch.utils.data.DataLoader(train_data,batch_size=batch_size) 
valid_loader =torch.utils.data.DataLoader(validation_data,batch_size=batch_size) 
test_loader =torch.utils.data.DataLoader(test_data,batch_size=batch_size) 
path = train(Encoder, train_loader, valid_loader, num_epochs=30, learning_rate=1e-2, batch_size=batch_size)

Epoch 1: Train loss: 943.3446169766513 |Validation loss: 965.1892237169989
Epoch 2: Train loss: 942.7918881503018 |Validation loss: 965.1797801050647
Epoch 3: Train loss: 942.7917836507162 |Validation loss: 965.1797801050647
Epoch 4: Train loss: 942.7917836507162 |Validation loss: 965.1797801050647
Epoch 5: Train loss: 942.7917836507162 |Validation loss: 965.1797801050647
Epoch 6: Train loss: 942.7917841131037 |Validation loss: 965.1797801050647
Epoch 7: Train loss: 942.7917841131037 |Validation loss: 965.1797801050647
Epoch 8: Train loss: 942.7917841131037 |Validation loss: 965.1797801050647
Epoch 9: Train loss: 942.7917836507162 |Validation loss: 965.1797801050647
Epoch 10: Train loss: 942.7917836507162 |Validation loss: 965.1797801050647
Epoch 11: Train loss: 942.7917831883286 |Validation loss: 965.1797801050647
Epoch 12: Train loss: 942.7917831883286 |Validation loss: 965.1797801050647
Epoch 13: Train loss: 942.7917831883286 |Validation loss: 965.1797801050647
Epoch 14: Train loss:

In [0]:
cat_index

{}